In [ ]:
from matplotlib import pyplot as plt
%matplotlib inline

In [1]:
from bokeh.plotting import figure, output_notebook, show
output_notebook()

Loading BokehJS ...

In [2]:
import glob

In [3]:
import numpy as np

In [4]:
from scipy.interpolate import interp1d
from scipy.interpolate import spline

In [5]:
from tqdm import tqdm

In [6]:
DQN_Rewards_Path = "/home/tabz/tmp/LogFiles/Maze_4_Logs/*Count*_Beta_0.01_uid*/logs/Episode_Rewards.txt"
DQN_Lengths_Path = "/home/tabz/tmp/LogFiles/Maze_4_Logs/*Count*_Beta_0.01_uid*/logs/Episode_Lengths.txt"
DQN_Rewards = []
DQN_Lengths = []
DQN_Lengths_C = []

In [7]:
for filename in glob.glob(DQN_Rewards_Path):
    run_logs = [0]
#     print(filename)
    with open(filename, "r") as f:
        for line in f:
            try:
                ep_r = float(line)
            except:
                continue
            run_logs.append(ep_r)
    DQN_Rewards.append(run_logs)

In [9]:
for filename in glob.glob(DQN_Lengths_Path):
    run_logs = [0]
#     print(filename)
    with open(filename, "r") as f:
        for line in f:
            try:
                ep_r = float(line)
            except:
                continue
            run_logs.append(ep_r)
    DQN_Lengths.append(run_logs)
    run_logs_c = np.cumsum(run_logs).astype(np.int32)
    DQN_Lengths_C.append(run_logs_c)

In [10]:
for lens, runs in zip(DQN_Lengths_C, DQN_Rewards):
    plt.plot(lens, runs)
plt.show()

NameError: name 'plt' is not defined

In [11]:
p = figure(width=500, height=500, y_range=(-1,3))
for lens, runs in zip(DQN_Lengths_C, DQN_Rewards):
    p.line(lens, runs)
show(p)

In [12]:
def plot_averages(names, t_maxs, colors, legends=None, directory="Logs/", step=10):
    plt.figure(figsize=(10,10))
    plt.ylim(-1, 3)
    for name, color, t_max in tqdm(zip(names, colors, t_maxs)):
        # TODO: Remove DQN from names
        DQN_Rewards_Path = directory + "*" + name + "*/logs/Episode_Rewards.txt"
        DQN_Lengths_Path = directory + "*" + name + "*/logs/Episode_Lengths.txt"
        DQN_Rewards = []
        DQN_Lengths = []
        DQN_Lengths_C = []
        for filename in glob.glob(DQN_Rewards_Path):
            run_logs = [0]
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Rewards.append(run_logs)
        for filename in glob.glob(DQN_Lengths_Path):
            run_logs = [0]
        #     print(filename)
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Lengths.append(run_logs)
            run_logs_c = np.cumsum(run_logs).astype(np.int32)
            DQN_Lengths_C.append(run_logs_c)
        DQN_Smoothed = []
        for times, rewards in zip(DQN_Lengths_C, DQN_Rewards):
#             linear = interp1d(times, rewards, kind="linear")
#             linear_rewards = linear([i for i in range(500000)])
#             DQN_Smoothed.append(linear_rewards)
            spline_rewards = spline(times, rewards, [i for i in range(0, t_max, step)])
            DQN_Smoothed.append(spline_rewards)
        DQN_Means = np.mean(DQN_Smoothed, axis=0)
        DQN_Stds = np.std(DQN_Smoothed, axis=0)
        plt.fill_between([i for i in range(0, t_max, step)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color=color, alpha=0.2, edgecolor="white")

        plt.plot([i for i in range(0, t_max, step)], DQN_Means, color=color)

    plt.title("Episode_Reward")
    plt.xlabel("T")
    plt.ylabel("Reward")
    if legends is not None:
        plt.legend(legends, bbox_to_anchor=(1.05, 1), loc=2)
    plt.show()

In [155]:
from bokeh.models import Legend
from bokeh.models.formatters import BasicTickFormatter
from bokeh.models import HoverTool, TapTool

In [181]:
def plot_averages_b(names, t_maxs, colors, legends, directory="Logs/", step=10):
#     plt.figure(figsize=(10,10))
    p = figure(width=1200, height=800, y_range=(-1.2, 3.2))
    p.toolbar_location = "above"
    p.yaxis.axis_label = "Episode Reward"
    p.xaxis.axis_label = "T"
    p.xaxis.formatter = BasicTickFormatter(use_scientific=False)
#     p = figure(width=800, height=500)
    lines = []
    alpha_lines = []
    for name, color, t_max, legend in tqdm(zip(names, colors, t_maxs, legends)):
        # TODO: Remove DQN from names
        DQN_Rewards_Path = directory + "*" + name + "*/logs/Episode_Rewards.txt"
        DQN_Lengths_Path = directory + "*" + name + "*/logs/Episode_Lengths.txt"
        DQN_Rewards = []
        DQN_Lengths = []
        DQN_Lengths_C = []
        for filename in glob.glob(DQN_Rewards_Path):
            run_logs = [0]
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Rewards.append(run_logs)
        for filename in glob.glob(DQN_Lengths_Path):
            run_logs = [0]
        #     print(filename)
            with open(filename, "r") as f:
                for line in f:
                    try:
                        ep_r = float(line)
                    except:
                        continue
                    run_logs.append(ep_r)
            DQN_Lengths.append(run_logs)
            run_logs_c = np.cumsum(run_logs).astype(np.int32)
            DQN_Lengths_C.append(run_logs_c)
        DQN_Smoothed = []
        for times, rewards in zip(DQN_Lengths_C, DQN_Rewards):
#             linear = interp1d(times, rewards, kind="linear")
#             linear_rewards = linear([i for i in range(500000)])
#             DQN_Smoothed.append(linear_rewards)
            spline_rewards = spline(times, rewards, [i for i in range(0, t_max, step)], order=1)
            DQN_Smoothed.append(spline_rewards)
        DQN_Means = np.mean(DQN_Smoothed, axis=0)
        DQN_Stds = np.std(DQN_Smoothed, axis=0)
        
#         plt.fill_between([i for i in range(0, t_max, step)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color=color, alpha=0.2, edgecolor="white")
        # Error bars
        xs = [i for i in range(0, t_max, step)]
        xs = xs + list(reversed(xs))
        ys = np.concatenate([DQN_Means - DQN_Stds, np.flip(DQN_Means + DQN_Stds, axis=0)])
        lls = p.patch(xs, ys, color=color, alpha=0.1)

        
#         for x, y, yerr in zip(range(0, t_max, step), DQN_Means, DQN_Stds):
#             lls = p.line([x, x], [y - yerr, y + yerr], color=color, alpha=0.1, hover_alpha=0.8)
#             alpha_lines.append(lls)
            

#         plt.plot([i for i in range(0, t_max, step)], DQN_Means, color=color)
        l = p.line([i for i in range(0, t_max, step)], DQN_Means, color=color, line_width=2)
        lines.append(l)
#         alpha_lines.append(lls)

#     plt.title("Episode_Reward")
#     plt.xlabel("T")
#     plt.ylabel("Reward")
#     if legends is not None:
#         plt.legend(legends)
#     plt.show()
#     p.line([0,1], [-1,3])
    new_leg = Legend(items=[(name, [line]) for name, line in zip(legends, lines)], location=(0,-30))
    p.add_layout(new_leg, "right")
#     p.add_tools(HoverTool(renderers=alpha_lines))
#     p.add_tools(TapTool(renderers=alpha_lines))
    show(p)

In [15]:
from matplotlib.cm import rainbow

In [16]:
rainbow([i for i in range(10)])

array([[ 0.5       ,  0.        ,  1.        ,  1.        ],
       [ 0.49215686,  0.01231966,  0.99998103,  1.        ],
       [ 0.48431373,  0.02463745,  0.99992411,  1.        ],
       [ 0.47647059,  0.0369515 ,  0.99982925,  1.        ],
       [ 0.46862745,  0.04925994,  0.99969645,  1.        ],
       [ 0.46078431,  0.06156091,  0.99952572,  1.        ],
       [ 0.45294118,  0.07385253,  0.99931706,  1.        ],
       [ 0.44509804,  0.08613294,  0.99907048,  1.        ],
       [ 0.4372549 ,  0.09840028,  0.99878599,  1.        ],
       [ 0.42941176,  0.11065268,  0.9984636 ,  1.        ]])

In [88]:
names = ["DQN"]
legends = ["DQN"]
t_maxs = [1000000]
for conv in [True, False]:
    names.append("Count*_Conv_{}*_Eps_1".format(conv))
    legends.append("Count_Conv_{}".format(conv))
    t_maxs.append(1000000)
    names.append("Count*_Conv_{}*_Eps_0.1".format(conv))
    legends.append("Count_Conv_{}_Epsilon".format(conv))
    t_maxs.append(1000000)
print(names)
print(t_maxs)
# colours = ["red", "#3366ff", "#66ff33", "yellow"]
colours = rainbow(np.linspace(0, 1, len(names)))

['DQN', 'Count*_Conv_True*_Eps_1', 'Count*_Conv_True*_Eps_0.1', 'Count*_Conv_False*_Eps_1', 'Count*_Conv_False*_Eps_0.1']
[1000000, 1000000, 1000000, 1000000, 1000000]


In [107]:
from bokeh.palettes import Set1_5 as rainbow_colours

In [182]:
plot_averages_b(names, t_maxs, rainbow_colours, legends, directory="/home/tabz/tmp/LogFiles/Maze_5_Logs/", step=10000)

5it [00:00, 99.53it/s]


In [ ]:
plot_averages(names, t_maxs, colours, legends, directory="/home/tabz/tmp/LogFiles/Maze_4_Logs/", step=1000)

In [ ]:
DQN_Smoothed = []
for times, rewards in zip(DQN_Lengths_C, DQN_Rewards):
    linear = interp1d(times, rewards, kind="linear")
    linear_rewards = linear([i for i in range(500000)])
    DQN_Smoothed.append(linear_rewards)

In [ ]:
DQN_Means = np.mean(DQN_Smoothed, axis=0)

In [ ]:
DQN_Stds = np.std(DQN_Smoothed, axis=0)

In [ ]:
plt.fill_between([i for i in range(500000)], DQN_Means - DQN_Stds, DQN_Means + DQN_Stds, color="red", alpha=0.3, edgecolor="white")

plt.plot([i for i in range(500000)], DQN_Means, color="red")

In [ ]:
len(DQN_Repeat)

In [ ]:
plt.plot(DQN)

In [ ]:
directories = ["Logs/"]

In [ ]:
for i in range(0, 10, 2):
    print(i)